In [45]:
# Import Dependencies
import pandas as pd
from datetime import datetime as dt
import sqlalchemy
from sqlalchemy import create_engine, func
import sqlite3

In [46]:
# create engine to Hospital_Boarding.sqlite
engine = create_engine("sqlite:///Hospital_Boarding.sqlite")
conn = sqlite3.connect('Hospital_Boarding.sqlite')

In [47]:
# Create admit_details dataframe be querying all records. Change date to from object to datetime.
admit_details_df = pd.read_sql("SELECT * FROM admit_details", conn)
admit_details_df['date'] = pd.to_datetime(admit_details_df['date'], format="%m/%d/%Y")

# Add columns for year and month for grouping. Change wait time from minutes to hours. Reorder columns
admit_details_df['year'] = admit_details_df['date']
admit_details_df['year_month'] = admit_details_df['date']
admit_details_df['year'] = admit_details_df['year'].dt.strftime('%Y')
admit_details_df['year_month'] = admit_details_df['year_month'].dt.strftime('%Y-%m')
admit_details_df['decision_to_discharge_time'] = admit_details_df['decision_to_discharge_time']/60
admit_details_df = admit_details_df[['hospital_code', 'year', 'year_month', 'date', 'disposition', 'decision_to_discharge_time']]
admit_details_df.head()

,hospital_code,year,year_month,date,disposition,decision_to_discharge_time
0,A,2021,2021-01,2021-01-01,Admitted,0.900000
1,A,2021,2021-01,2021-01-01,Admitted,0.850000
2,A,2021,2021-01,2021-01-01,Admitted,1.150000
3,A,2021,2021-01,2021-01-01,Admit Obs,1.866667
4,A,2021,2021-01,2021-01-01,Admitted,2.033333


In [48]:
# Create hospitals dataframe from database
hospitals_df = pd.read_sql("SELECT * FROM hospitals", conn)

In [49]:
# Create data for month-by-month bar chart
# Wait time month-by-month for each hospital
wait_time_h = admit_details_df.groupby(['hospital_code', 'year_month'])['decision_to_discharge_time'].mean().reset_index()
wait_time_h = wait_time_h.rename(columns = {'decision_to_discharge_time':'mean_wait'})

In [50]:
# Wait time month-by-month for all hospitals
wait_time_a = admit_details_df.groupby(['year_month'])['decision_to_discharge_time'].mean().reset_index()
wait_time_a = wait_time_a.rename(columns = {'decision_to_discharge_time':'mean_wait'})
wait_time_a['hospital_code'] = 'all'
wait_time_a = wait_time_a[['hospital_code', 'year_month', 'mean_wait']]

In [51]:
# Concantenate wait time month-by-month dataframes into one dataframe, and merge location data from hospitals
wait_time_month = pd.concat([wait_time_h, wait_time_a], axis = 0)
wait_time_month_location = pd.merge(wait_time_month, hospitals_df, on='hospital_code', how = 'outer')
wait_time_month_location.head()

,hospital_code,year_month,mean_wait,City,Latitude,Longitude
0,A,2021-01,2.173894,Bemidji,47.478542,-94.890787
1,A,2021-02,1.925018,Bemidji,47.478542,-94.890787
2,A,2021-03,1.880336,Bemidji,47.478542,-94.890787
3,A,2021-04,2.578033,Bemidji,47.478542,-94.890787
4,A,2021-05,2.317561,Bemidji,47.478542,-94.890787


In [52]:
# Create data for month-by-day wait times for each hospital
wait_time_day_h = admit_details_df.groupby(['hospital_code', 'year_month', 'date'])['decision_to_discharge_time']\
    .mean().reset_index()
wait_time_day_h = wait_time_day_h.rename(columns = {'decision_to_discharge_time':'mean_wait'})

In [53]:
# Create data for month-by-day wait times for all hospitals
wait_time_day_a = admit_details_df.groupby(['year_month', 'date'])['decision_to_discharge_time'].mean().reset_index()
wait_time_day_a = wait_time_day_a.rename(columns = {'decision_to_discharge_time':'mean_wait'})
wait_time_day_a['hospital_code'] = 'all'
wait_time_day_a = wait_time_day_a[['hospital_code', 'year_month', 'date', 'mean_wait']]

In [54]:
# Concantenate wait time month-by-month dataframes into one dataframe for use in JS
wait_time_day = pd.concat([wait_time_day_h, wait_time_day_a], axis = 0)
wait_time_day_location = pd.merge(wait_time_day, hospitals_df, on='hospital_code', how = 'outer')
wait_time_day_location.head()


,hospital_code,year_month,date,mean_wait,City,Latitude,Longitude
0,A,2021-01,2021-01-01,1.580208,Bemidji,47.478542,-94.890787
1,A,2021-01,2021-01-02,1.573333,Bemidji,47.478542,-94.890787
2,A,2021-01,2021-01-03,2.595652,Bemidji,47.478542,-94.890787
3,A,2021-01,2021-01-04,1.722727,Bemidji,47.478542,-94.890787
4,A,2021-01,2021-01-05,2.394253,Bemidji,47.478542,-94.890787


In [55]:
# Create data for pie chart
# Change Admit Obs to Admitted to simplify later code
admit_trans_detail = admit_details_df
admit_trans_detail['disposition'] = admit_trans_detail['disposition'].replace({'Admit Obs': "Admitted"})

In [56]:
# Create dataframe that groups by disposition and format to concantenate to other dataframes
disp_all = admit_trans_detail.groupby('disposition')['date'].count().reset_index()
disp_all = disp_all.rename(columns = {'date':"count"})
disp_all['hospital_code'] = 'all'
disp_all['year'] = '--'
disp_all = disp_all[['hospital_code', 'year', 'disposition', 'count']]

In [57]:
# Create dataframe that groups by hospital code and disposition with counts. Format to concantenate
disp_h = admit_trans_detail.groupby(['hospital_code','disposition'])['date'].count().reset_index()
disp_h = disp_h.rename(columns = {'date':"count"})
disp_h['year'] = '--'
disp_h = disp_h[['hospital_code', 'year', 'disposition', 'count']]

In [59]:
# Create dataframe that groups by year for all hospitals and format to concantenate
disp_all_y = admit_trans_detail.groupby(['year','disposition'])['date'].count().reset_index()
disp_all_y = disp_all_y.rename(columns = {'date':"count"})
disp_all_y['hospital_code'] = 'all'
disp_all_y = disp_all_y[['hospital_code', 'year', 'disposition', 'count']]

In [60]:
# Create dataframe that groups by hospital code and year and format to concantenate
disp_h_y = admit_trans_detail.groupby(['hospital_code', 'year', 'disposition'])['date'].count().reset_index()
disp_h_y = disp_h_y.rename(columns = {'date':"count"})
disp_h_y = disp_h_y[['hospital_code', 'year', 'disposition', 'count']]

In [61]:
# Concantenate disposition dataframes into one dataframe to pass to JS
disposition = pd.concat([disp_all, disp_h, disp_all_y, disp_h_y], axis = 0)
disposition.head()

,hospital_code,year,disposition,count
0,all,--,Admitted,227339
1,all,--,Transferred,20342
0,A,--,Admitted,14317
1,A,--,Transferred,1227
2,B,--,Admitted,3389


In [62]:
#  Create views
c = conn.cursor()

c.execute('''DROP TABLE IF EXISTS wait_times_month ''')

c.execute('''CREATE TABLE wait_times_month (
    record_id INTEGER NOT NULL PRIMARY KEY,
    hospital_code VARCHAR  NOT NULL,
    year_month TEXT  NOT NULL,
    mean_wait DECIMAL,
    City VARCHAR,
    Latitude DECIMAL,
    Longitude DECIMAL)
    ''')

c.execute('''DROP TABLE IF EXISTS wait_times_day ''')

c.execute('''CREATE TABLE wait_times_day (
    record_id INTEGER NOT NULL PRIMARY KEY,
    hospital_code VARCHAR  NOT NULL,
    year_month TEXT  NOT NULL,
    date TEXT   NOT NULL,
    mean_wait DECIMAL,
    City VARCHAR,
    Latitude DECIMAL,
    Longitude DECIMAL)
    ''')

c.execute('''DROP TABLE IF EXISTS admit_v_transfer ''')

c.execute('''CREATE TABLE admit_v_transfer (
    record_id INTEGER NOT NULL PRIMARY KEY,
    hospital_code VARCHAR  NOT NULL,
    year TEXT,
    disposition TEXT   NOT NULL,
    count DECIMAL   NOT NULL)
    ''')


In [63]:
# Use pandas to load DataFrame into database

wait_time_month_location.to_sql('wait_times_month', con=engine, if_exists='append', index=False)
wait_time_day_location.to_sql('wait_times_day', con=engine, if_exists='append', index=False)
disposition.to_sql('admit_v_transfer', con=engine, if_exists='append', index=False)


In [64]:
#  Close the connection
conn.close()